In [2]:
from splinter import Browser
import time
import requests
from PIL import Image
import os
import csv
import io
import hashlib
#browser = Browser('chrome', executable_path, headless=False)

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}


In [4]:
def scroll_to_end(driver,sleep_interactions):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_interactions)

        #selenium.time.sleep(sleep_between_interactions)    

In [7]:
def fetch_images(path, query,s_interactions,max_links_to_fetch):

    #open chrome driver
    wd = Browser('chrome', path, headless=False) # defaults to firefox

    #create query string
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    #search for the query
    wd.visit(search_url.format(q=query))

    
    #parameter set up
    image_urls = []
    image_count = 0
    results_start = 0

    while image_count < max_links_to_fetch:
        #scroll_to_end(wd,s_interactions)

        # get all image thumbnail results
        thumbnail_results = wd.find_by_css("img.Q4LuWd")
        number_results = len(thumbnail_results)
        image_count += 1
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        print(image_count)

        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(s_interactions)
            except Exception:
                #wd.quit()
                continue

            # extract image urls    
            actual_images = wd.find_by_css('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image['src'] and 'http' in actual_image['src']:
                    image_urls.append(actual_image['src'])

    #         image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)
        wd.quit()

    return image_urls


In [5]:
def fetch_images(path, query,s_interactions,max_links_to_fetch):

    #open chrome driver
    wd = Browser('chrome', path, headless=False) # defaults to firefox

    #create query string
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    #search for the query
    wd.visit(search_url.format(q=query))

    
    #parameter set up
    image_urls = []
    image_count = 0
    results_start = 0

    while image_count < max_links_to_fetch:
        #scroll_to_end(wd,s_interactions)

        # get all image thumbnail results
        thumbnail_results = wd.find_by_css("img.Q4LuWd")
        number_results = len(thumbnail_results)
        image_count += 1
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        print(image_count)

        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(s_interactions)
            except Exception:
                #wd.quit()
                continue

            # extract image urls    
            actual_images = wd.find_by_css('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image['src'] and 'http' in actual_image['src']:
                    image_urls.append(actual_image['src'])

    #         image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)}, {query} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            wd.quit()
            time.sleep(3)
            

        # move the result startpoint further down
        #results_start = len(thumbnail_results)
        wd.quit()

    return image_urls


In [8]:
fetch_images(executable_path,"Dogs",60,1)

working
Found: 100 search results. Extracting links from 0:100
1
Found: 1 image links, done!


{'https://i.ytimg.com/vi/MPV2METPeJU/maxresdefault.jpg'}

In [6]:
def persist_image(folder_path,url,img_name):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')

        picture_path = img_name.replace(" ", "-").lower()
        #print(picture_path)

        #file_path = os.path.join(folder_path,picture_path + '.jpg')
        #file_path = os.path.join(folder_path,img_name + '.jpg')


        if img_name is None:
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
            print(img_name)
        else:
            file_path = os.path.join(folder_path,picture_path + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [13]:
persist_image("./test", "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/21_Savage_2018.jpg/1200px-21_Savage_2018.jpg","21_savage")


SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/21_Savage_2018.jpg/1200px-21_Savage_2018.jpg - as ./test\21_savage.jpg


In [13]:
file_path = os.path.join("hiphop2.csv")
rapper_urls= []
dummy_image = "https://upload.wikimedia.org/wikipedia/en/thumb/a/a6/Pok%C3%A9mon_Pikachu_art.png/220px-Pok%C3%A9mon_Pikachu_art.png"
rappers_not_found = []
with open(file_path) as rapper_data:
    reader = csv.reader(rapper_data)
    header = next(reader)

    for row in reader:
        #print(row[3])
        rapper_name = row[4] 
        rapper_search = f'{row[4]} rapper' 
        rapper_url = fetch_images(executable_path,rapper_name,3,1)
        #print(rapper_url)
        try:
            rapper_urls.append(rapper_url[0])
            #persist_image("./assets/images",rapper_url[0],rapper_name)

        except Exception as e:
            print(e)
            rapper_urls.append(dummy_image)
            rappers_not_found.append(rapper_name)



#print(rapper_urls)

Found: 48 search results. Extracting links from 0:48
1
Found: 1, 21 Savage image links, done!
Found: 48 search results. Extracting links from 0:48
1
Found: 1, 2 Chainz image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, 2Pac image links, done!
Found: 48 search results. Extracting links from 0:48
1
Found: 1, 50 Cent image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Ab-Soul image links, done!
Found: 48 search results. Extracting links from 0:48
1
Found: 1, Action Bronson image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Aesop Rock image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 0 image links, looking for more ...
list index out of range
Found: 49 search results. Extracting links from 0:49
1
Found: 1, A$AP Ferg image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Atmosphere image links, done!
Found: 49 search results. Extractin

WebDriverException: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=88.0.4324.150)


In [33]:
file_path = os.path.join("hiphop2.csv")
rapper_urls= []
dummy_image = "https://upload.wikimedia.org/wikipedia/en/thumb/a/a6/Pok%C3%A9mon_Pikachu_art.png/220px-Pok%C3%A9mon_Pikachu_art.png"
#rappers_not_found = []
with open(file_path) as rapper_data:
    reader = csv.reader(rapper_data)
    #header = next(reader)

    #read a specific row
    reader_list = list(reader)

    for row in reader_list[92:]:
        #print(row[3])
        rapper_name = row[4] 
        rapper_search = f'{row[4]} rapper' 
        rapper_url = fetch_images(executable_path,rapper_name,3,1)
        #print(rapper_url)
        try:
            rapper_urls.append({"url":rapper_url[0],"name":rapper_name})
            #persist_image("./assets/images",rapper_url[0],rapper_name)

        except Exception as e:
            print(e)
            rapper_urls.append(dummy_image)
            rappers_not_found.append(rapper_name)



#print(rapper_urls)

Found: 48 search results. Extracting links from 0:48
1
Found: 1, Lil' Kim image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Lil Uzi Vert image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Lil Wayne image links, done!
Found: 48 search results. Extracting links from 0:48
1
Found: 1, Lil Yachty image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, LL Cool J image links, done!
Found: 48 search results. Extracting links from 0:48
1
Found: 1, Logic image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Ludacris image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Lupe Fiasco image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Mac Miller image links, done!
Found: 49 search results. Extracting links from 0:49
1
Found: 1, Mac Dre image links, done!
Found: 48 search results. Extracting links from 0:48
1
Found:

In [15]:
len(rapper_urls)

85

In [34]:
rappers_not_found

['Big Daddy Kane',
 'Big Daddy Kane',
 'GZA',
 'Killah Priest',
 'KRS-One',
 'Meek Mill',
 'Migos',
 'NF',
 'Rick Ross',
 'ScHoolboy Q',
 '$uicideboy$',
 'Wu-Tang Clan']

In [35]:
for rapper in rapper_urls:
    try:
        persist_image("./assets/images",rapper['url'],rapper['name'])

    except Exception as e:
        print(e)

    

SUCCESS - saved https://observer.com/wp-content/uploads/sites/2/2018/06/gettyimages-847158018.jpg?quality=80 - as ./assets/images\lil'-kim.jpg
SUCCESS - saved https://www.rollingstone.com/wp-content/uploads/2021/02/lil-uzi-vert-diamond.jpg - as ./assets/images\lil-uzi-vert.jpg
SUCCESS - saved https://i.guim.co.uk/img/media/282ad68589d12c94a9439aad73181466d75c54eb/0_150_3600_2160/master/3600.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=c8e2be5dd59936e726bdeb8e5acf1b13 - as ./assets/images\lil-wayne.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/5/5e/Lil_Yachty_cropped.jpg - as ./assets/images\lil-yachty.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/6/67/LL_Cool_J_in_2017.jpg - as ./assets/images\ll-cool-j.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/f/f0/Logic_Grandoozy_2018_%2845208197292%29.jpg - as ./assets/images\logic.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/c/c2/Ludacris_2008.j

In [29]:
rappers_not_found

['Ab-Soul', 'Aesop Rock', 'Big Daddy Kane', 'A Boogie wit da Hoodie']

In [ ]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [15]:
file_path = os.path.join("hiphop2.csv")
rapper_urls= []
with open(file_path) as rapper_data:
    reader = csv.reader(rapper_data)
    header = next(reader)

    for row in reader:
        #print(row[3])
        rapper_name = row[4] 
        rapper_search = f'{row[4]} rapper' 
        rapper_url = fetch_images(executable_path,rapper_name,3,1)
        #print(rapper_url)
        try:
            persist_image("./assets/images",rapper_url[0],rapper_name)

        except Exception as e:
            print(e)
            wd.quit()


        #rapper_urls.append(rapper_url)

#print(rapper_urls)




Found: 100 search results. Extracting links from 0:100
1
Found: 1 image links, done!
SUCCESS - saved https://ichef.bbci.co.uk/news/976/cpsprodpb/11DF7/production/_105470237_mediaitem105470234.jpg - as ./assets/images\21-savage.jpg
Found: 100 search results. Extracting links from 0:100
1
Found: 1 image links, done!
SUCCESS - saved https://i.ytimg.com/vi/e2QKlmMT8II/maxresdefault.jpg - as ./assets/images\2-chainz.jpg
Found: 100 search results. Extracting links from 0:100
1
Found: 1 image links, done!
SUCCESS - saved https://i.guim.co.uk/img/media/c1f430159c3e7e58c9b46098efde3dee7ba7d830/0_196_2515_1509/master/2515.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f554bca9c12ca49c79264bce18b42472 - as ./assets/images\2pac.jpg
Found: 100 search results. Extracting links from 0:100
1
Found: 1 image links, done!
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/a/a3/50_Cent_AKA_Curtis_Jackson_%28cropped%29.jpg - as ./assets/images\50-cent.jpg
Found: 100 search results